# Qubit-algebra-based measurement optimization

The measurement optimization techniques introduced here are applicable to a general observable, but for concreteness, we focus on measuring the expectation value of the molecular electronic Hamiltonian. Measuring the expectation value of $\hat{H}$ is an important subroutine and a bottleneck of the variational quantum eigensolver \[see [Phys. Rev. Research **4**, 033154 (2022)](https://journals.aps.org/prresearch/abstract/10.1103/PhysRevResearch.4.033154)\].

Qubit-algebra-based techniques first present the molecular electronic Hamiltonian in its qubit representation
$$
\hat{H} = \sum_{j} c_{j} \hat{P}_{j},
$$
obtained by, e.g., Jordan&ndash;Wigner transformation. Then, 
the expectation value of the qubit Hamiltonian is measured by grouping $\hat{P}_{j}$ into mutually commutative groups since each such group can be measured simultaneously \[[J. Chem. Theory Comput. **16**(4), 2400–2409 (2020)](https://doi.org/10.1021/acs.jctc.0c00008)\]. The number of measurements required to obtain the expectation value of $\hat{H}$ depends on how we group the Pauli products and how many measurements (shots) we assign to each group (measurement allocation).

In this tutorial, we introduce two measurement optimization schemes available within Tequila:

1. Sorted insertion (SI) [Quantum **5**, 385 (2021)](https://doi.org/10.22331/q-2021-01-20-385) 
2. Iterative coefficient splitting (ICS) [npj Quantum Inf. **9**, 14 (2023)](https://www.nature.com/articles/s41534-023-00683-y) 


## Common to both SI and ICS
### Setting up the test Hamiltonian and covariance dictionary:
To start, both SI and ICS require approximate covariances between $\hat{P}_{j}$'s in $\hat{H}$. In SI, these covariances are used to optimize the measurement allocation. In ICS, they are additionally used for the optimization of Pauli product groups. Because quantum wavefunction is only available on the quantum computer, we use a classically efficient proxy \[e.g., Hartree&ndash;Fock or configuration interaction singles and doubles (CISD) wavefunction\] to approximate the variances. Here we use the CISD wavefunction.

For concreteness, we use H$_4$ in the STO-3G basis for demonstration.

In [1]:
# Needs pyscf or psi4 installed:
# pip install pyscf
# tequila version needs to be > 1.8.3 or from devel branch
import tequila as tq
from tequila.hamiltonian import QubitHamiltonian, PauliString
from tequila.grouping.binary_rep import BinaryPauliString, BinaryHamiltonian
from tequila.grouping.fermionic_functions import n_elec
from tequila.grouping.fermionic_methods import get_wavefunction
import numpy as np

def prepare_test_hamiltonian():
    '''
    Return a test hamiltonian.
    '''
    trafo = "JordanWigner"
    mol = tq.chemistry.Molecule(
                            geometry="H 0.0 0.0 0.0 \n H 0.0 0.0 1. \n H 0.0 0.0 2. \n H 0.0 0.0 3.",
                            basis_set="sto3g",
                            transformation=trafo,
                            backend='pyscf'
                                 )
    H = mol.make_hamiltonian()
    Hbin = BinaryHamiltonian.init_from_qubit_hamiltonian(H)
    _, psis_appr = get_wavefunction(H.to_openfermion(), "cisd", "h4", n_elec("h4"), save=False)
    return mol, H, Hbin, psis_appr[0], len(Hbin.binary_terms) - 1

def prepare_cov_dict(H, psi_appr):
    '''
    Return the covariance dictionary containing Cov(P1, P2). 
    In a practical calculation, this covariance dictionary would be built from
    a Hartree-Fock or configuration interaction singles and doulbes (CISD) 
    wavefunction. Here, we use the CISD wavefunction.
    '''
    terms = H.binary_terms
    cov_dict = {}
    wfn0 = tq.QubitWaveFunction(psi_appr)
    for idx, term1 in enumerate(terms):
        for term2 in terms[idx:]:
            pw1 = BinaryPauliString(term1.get_binary(), 1.0)
            pw2 = BinaryPauliString(term2.get_binary(), 1.0)
            op1 = QubitHamiltonian.from_paulistrings(pw1.to_pauli_strings())
            op2 = QubitHamiltonian.from_paulistrings(pw2.to_pauli_strings())
            if pw1.commute(pw2):
                prod_op = op1 * op2
                cov_dict[(term1.binary_tuple(), term2.binary_tuple())] = wfn0.inner(prod_op(wfn0)) - wfn0.inner(op1(wfn0)) * wfn0.inner(op2(wfn0))
    return cov_dict

mol, H, Hbin, psi_appr, n_paulis = prepare_test_hamiltonian()
print("Number of Pauli products to measure: {}".format(n_paulis))
cov_dict = prepare_cov_dict(Hbin, psi_appr)

converged SCF energy = -2.09854593699776
Number of Pauli products to measure: 184


## Sorted Insertion

To employ SI, simply define an "options" dictionary as follows. Note that the user can choose between fully commuting (FC) and qubit-wise commuting (QWC) fragments. FC yields fewer number of quantum measurements. However, a more complicated quantum circuit is needed to rotate an FC group into measurable Pauli-$\hat{z}$ operators. For further details, including numerical results, see [npj Quantum Inf. **9**, 14 (2023)](https://www.nature.com/articles/s41534-023-00683-y) and [J. Phys. Chem. A  **126**, 7007 (2022)](https://pubs.acs.org/doi/full/10.1021/acs.jpca.2c04726).

In [2]:
options = {"method":"si", "condition": "fc", "cov_dict":cov_dict}

### Running SI
One can obtain the groups of commuting Pauli products and optimal measurement allocation for each group by running:

In [3]:
commuting_parts, suggested_sample_size = Hbin.commuting_groups(options=options)
print("Number of SI groups: {}".format(len(commuting_parts)))
for idx, part in enumerate(commuting_parts):
    print("Group # {}: Mutually commuting? {}".format(idx+1, part.is_commuting()))
print("Suggested samples:", suggested_sample_size)

Number of SI groups: 9
Group # 1: Mutually commuting? True
Group # 2: Mutually commuting? True
Group # 3: Mutually commuting? True
Group # 4: Mutually commuting? True
Group # 5: Mutually commuting? True
Group # 6: Mutually commuting? True
Group # 7: Mutually commuting? True
Group # 8: Mutually commuting? True
Group # 9: Mutually commuting? True
Suggested samples: [0.27906757 0.05827309 0.19752505 0.19658682 0.05167209 0.02181115
 0.02181115 0.09466249 0.07859059]


### Estimating the number of required measurements in SI
In the following, we estimate the number of measurements (in millions) required to obtain the Hamiltonian expectation value with a millihartree accuracy. For this estimation, we suppose that one is interested in measuring $\langle \mathrm{FCI} | \hat{H} | \mathrm{FCI} \rangle$ because all succesful VQE algorithms should converge close to the FCI solution. To evaluate the measurement cost, we use Eq. (4) from [J. Chem. Theory Comput. **18**, 7394 (2022)](https://pubs.acs.org/doi/pdf/10.1021/acs.jctc.2c00837).

In [4]:
var_metric = 0.
fci_energies, psis_fci = get_wavefunction(H.to_openfermion(), "fci", "h4", n_elec("h4"), save=False)
wf_fci = tq.QubitWaveFunction(psis_fci[0])
for idx, part in enumerate(commuting_parts):
    op = part.to_qubit_hamiltonian()
    prod_op = op * op
    var_part = wf_fci.inner(prod_op(wf_fci)) - wf_fci.inner(op(wf_fci)) ** 2 
    var_metric += var_part/suggested_sample_size[idx]
print("Required number of measurements (in miilions): {}".format(np.real_if_close(var_metric)))

Required number of measurements (in miilions): 1.0156030017170306


## Iterative coefficient splitting

To employ ICS, simply define an "options" dictionary as follows. 
Like in SI, the user can choose between fully commuting (FC) and qubit-wise commuting (QWC) fragments.

In [5]:
options_ics = {"method":"ics", "condition": "fc", "cov_dict":cov_dict}

### Running ICS
One can obtain the groups of commuting Pauli products and optimal number of measurements for each group by running:

In [6]:
commuting_parts_ics, suggested_sample_size_ics = Hbin.commuting_groups(options=options_ics)
print("Number of ICS groups: {}".format(len(commuting_parts_ics)))
for idx, part in enumerate(commuting_parts_ics):
    print("Group # {}: Mutually commuting?  {}".format(idx+1, part.is_commuting()))
    print("\t   Is group same as SI? {}".format(part == commuting_parts[idx]))

Number of ICS groups: 9
Group # 1: Mutually commuting?  True
	   Is group same as SI? False
Group # 2: Mutually commuting?  True
	   Is group same as SI? False
Group # 3: Mutually commuting?  True
	   Is group same as SI? False
Group # 4: Mutually commuting?  True
	   Is group same as SI? False
Group # 5: Mutually commuting?  True
	   Is group same as SI? False
Group # 6: Mutually commuting?  True
	   Is group same as SI? False
Group # 7: Mutually commuting?  True
	   Is group same as SI? False
Group # 8: Mutually commuting?  True
	   Is group same as SI? False
Group # 9: Mutually commuting?  True
	   Is group same as SI? False


### Estimating the number of required measurements in ICS
The optimization of Pauli product groups in ICS means that the number of required measurements in ICS is lower than that in SI:

In [7]:
var_metric_ics = 0.
for idx, part in enumerate(commuting_parts_ics):
    op = part.to_qubit_hamiltonian()
    prod_op = op * op
    var_part = wf_fci.inner(prod_op(wf_fci)) - wf_fci.inner(op(wf_fci)) ** 2 
    var_metric_ics += var_part/suggested_sample_size_ics[idx]
print("Required number of measurements (in miilions): {}".format(np.real_if_close(var_metric_ics)))
print("The number of measurements is {:.2f} times lower than that in SI".format(np.real(var_metric/var_metric_ics)))

Required number of measurements (in miilions): 0.7074154405316988
The number of measurements is 1.44 times lower than that in SI


## Measuring the expectation value using a quantum circuit
Now let us use Tequila to measure the expectation value of a test wavefunction (wfn) on a quantum circuit. While the demonstration is only for ICS, it is easily transferrable to SI by simply replacing "options_ics" with its SI counterpart (by specifying {"method":"si"}).

In [8]:
U = mol.make_ansatz(name="SPA", edges=[(0,1), (2,3)])
E = tq.ExpectationValue(H=H, U=U)
result = tq.minimize(E, silent=True)
wfn = tq.simulate(U, variables=result.variables)

print("=========================================")
print("ICS measurement scheme")
print("=========================================")
e_ics = tq.ExpectationValue(H=Hbin.to_qubit_hamiltonian(), U=U, optimize_measurements=options_ics)
print(e_ics)

#Print Benchmark Energy (unnecessary in real measurement).
result_ics = tq.simulate(e_ics, result.variables)
print("Benchmark Energy:", result_ics)

compiled_ics = tq.compile(e_ics)
# auto-100000 means automatically allocate 100000 samples (shots) 
# between measurable fragments.
exp_ics = compiled_ics(result.variables, samples="auto-700000")

print("Energy measured with ICS:", exp_ics)

ICS measurement scheme
Objective with 9 unique expectation values
total measurements = 9
variables          = [((0, 1), 'D', None), ((2, 3), 'D', None)]
types              = not compiled
Benchmark Energy: -1.4066483974456787
Energy measured with ICS: -1.4041930437088013
